In [5]:
import re, requests, json, time, random
from bs4 import BeautifulSoup
import pandas as pd
import os

BASE = "https://www.10000recipe.com"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

# ✅ cat4 분류
CAT4_MAP = {
    63: "밑반찬", 56: "메인반찬", 54: "국/탕", 55: "찌개",
    53: "면/만두", 52: "밥/죽/떡", 61: "퓨전", 65: "양식", 64: "샐러드", 68: "스프", 62: "기타"
}

# ✅ 공통 파서
def get_soup(url):
    r = requests.get(url, headers=HEADERS, timeout=15)
    r.raise_for_status()
    return BeautifulSoup(r.text, "html.parser")

# ✅ 페이지에서 조회수 10만 이상 레시피 ID만 추출
def get_recipe_ids_over100k(cat4, cat_name):
    ids = []
    page = 1
    prev_ids = set()

    while True:
        url = f"{BASE}/recipe/list.html?cat4={cat4}&order=reco&page={page}"
        soup = get_soup(url)
        cards = soup.select("ul.common_sp_list_ul li")
        if not cards:
            print(f"[{cat_name}] 페이지 없음 — 누적 {len(ids)}개")
            break

        new_ids = []
        for li in cards:
            # recipe_id 추출
            link = li.select_one("a.common_sp_link")
            if not link or "href" not in link.attrs:
                continue
            m = re.search(r"/recipe/(\d+)", link["href"])
            if not m:
                continue
            recipe_id = int(m.group(1))

            # 조회수 텍스트 추출
            views_el = li.select_one("span.common_sp_caption_buyer")
            if not views_el:
                continue
            text = views_el.get_text(strip=True)
            text = text.replace("조회수", "").replace(",", "").replace(" ", "")
            if "만" in text:
                try:
                    views = float(text.replace("만", "")) * 10000
                except:
                    continue
            else:
                try:
                    views = int(re.sub(r"\D", "", text))
                except:
                    continue

            # ✅ 필터링: 조회수 5만 이상
            if views >= 50000:
                new_ids.append(recipe_id)

        ids_set = set(new_ids)
        if not new_ids or ids_set == prev_ids:
            print(f"[{cat_name}] 중단 (page {page}) — 새로운 레시피 없음")
            break

        prev_ids = ids_set
        ids.extend(new_ids)
        print(f"[{cat_name}] page {page} 완료 — 이번 {len(new_ids)}개 / 누적 {len(ids)}개")
        page += 1
        time.sleep(random.uniform(1.0, 2.0))

    return list(dict.fromkeys(ids))

# ✅ 상세 페이지 파서 (이전 버전과 동일)
def parse_recipe(recipe_id, cat4=None):
    url = f"{BASE}/recipe/{recipe_id}"
    try:
        soup = get_soup(url)
    except Exception as e:
        print(f"❌ {recipe_id} 불러오기 실패: {e}")
        return None

    title = soup.select_one("div.view2_summary h3, h3.view2_title")
    title = title.get_text(strip=True) if title else None

    summary_el = soup.select_one("#relationGoods > div.best_tit > b:nth-child(1)")
    summary = summary_el.get_text(strip=True) if summary_el else None

    # 인분
    serving_el = soup.select_one(
        "#contents_area_full div.view2_summary_info span.view2_summary_info1"
    )
    servings = None
    if serving_el:
        nums = re.findall(r"\d+", serving_el.get_text(strip=True))
        if nums:
            servings = int(nums[0])

    # 조리시간
    time_el = soup.select_one(
        "#contents_area_full div.view2_summary_info span.view2_summary_info2"
    )
    cook_time = None
    if time_el:
        nums = re.findall(r"\d+", time_el.get_text(strip=True))
        if nums:
            cook_time = int(nums[0])

    # 난이도
    level_el = soup.select_one(
        "#contents_area_full div.view2_summary_info span.view2_summary_info3"
    )
    level = level_el.get_text(strip=True) if level_el else None

    type_nm = CAT4_MAP.get(cat4, None)

    # 재료
    ingredients = {}
    for li in soup.select(".ready_ingre3 li, .ingre_list li"):
        text = li.get_text(" ", strip=True)
        if not text:
            continue  # 공백, 빈 항목 건너뛰기

        clean_text = text.replace("구매", "").strip()
        parts = clean_text.split()

        # parts가 비었을 경우 대비
        if not parts:
            continue

        name = parts[0]
        amount = " ".join(parts[1:]) if len(parts) > 1 else ""
        ingredients[name] = amount

    # 조리 단계
    steps = []
    for i, el in enumerate(soup.select("#stepDiv .media-body, .view_step_cont"), 1):
        txt = el.get_text(" ", strip=True)
        if txt:
            steps.append(f"{i}. {txt}")

    # 팁
    tips = []
    for i in range(1, len(steps) + 1):
        tip_el = soup.select_one(f"#stepdescr{i} > p")
        tip = tip_el.get_text(strip=True) if tip_el else ""
        tips.append(tip)

    return {
        "RECIPE_ID": recipe_id,
        "RECIPE_NM_KO": title,
        "SUMRY": summary,
        "SERVINGS": servings,
        "TY_NM": type_nm,
        "COOKING_TIME": cook_time,
        "LEVEL_NM": level,
        "INGREDIENT_FULL": json.dumps(ingredients, ensure_ascii=False).replace('"', "'"),
        "STEP_TEXT": json.dumps(steps, ensure_ascii=False),
        "STEP_TIP": json.dumps(tips, ensure_ascii=False),
    }

# ✅ 실행부
if __name__ == "__main__":
    for cat4, cat_name in CAT4_MAP.items():
        print(f"\n🌿 [START] {cat_name} (cat4={cat4}) 수집 시작")
        ids = get_recipe_ids_over100k(cat4, cat_name)
        print(f"[{cat_name}] 조회수 10만 이상 레시피 {len(ids)}개 발견")

        data_list = []
        for rid in ids:
            data = parse_recipe(rid, cat4=cat4)
            if data:
                data_list.append(data)
            time.sleep(random.uniform(1.0, 1.5))

        if data_list:
            df = pd.DataFrame(data_list)
            # 파일명에 사용할 수 없는 문자(예: / \ : * ? " < > |)를 언더스코어로 대체
            safe_cat = re.sub(r'[<>:"/\\|?*]', '_', cat_name)
            filename = f"recipes_over100k2_{safe_cat}.csv"
            out_path = os.path.join(os.getcwd(), filename)
            df.to_csv(out_path, index=False, encoding="utf-8-sig")
            print(f"💾 {cat_name} 완료 — {len(df)}개 저장 → {out_path}")
        else:
            print(f"⚠️ {cat_name}: 수집된 데이터 없음")

        time.sleep(2.0)

    print("\n🎯 전체 cat4 수집 완료 — 모든 조회수 10만 이상 레시피 저장 완료!")



🌿 [START] 밑반찬 (cat4=63) 수집 시작
[밑반찬] page 1 완료 — 이번 8개 / 누적 8개
[밑반찬] page 2 완료 — 이번 10개 / 누적 18개
[밑반찬] page 3 완료 — 이번 19개 / 누적 37개
[밑반찬] page 4 완료 — 이번 17개 / 누적 54개
[밑반찬] page 5 완료 — 이번 21개 / 누적 75개
[밑반찬] page 6 완료 — 이번 22개 / 누적 97개
[밑반찬] page 7 완료 — 이번 20개 / 누적 117개
[밑반찬] page 8 완료 — 이번 12개 / 누적 129개
[밑반찬] page 9 완료 — 이번 20개 / 누적 149개
[밑반찬] page 10 완료 — 이번 15개 / 누적 164개
[밑반찬] page 11 완료 — 이번 17개 / 누적 181개
[밑반찬] page 12 완료 — 이번 16개 / 누적 197개
[밑반찬] page 13 완료 — 이번 11개 / 누적 208개
[밑반찬] page 14 완료 — 이번 22개 / 누적 230개
[밑반찬] page 15 완료 — 이번 14개 / 누적 244개
[밑반찬] page 16 완료 — 이번 19개 / 누적 263개
[밑반찬] page 17 완료 — 이번 13개 / 누적 276개
[밑반찬] page 18 완료 — 이번 11개 / 누적 287개
[밑반찬] page 19 완료 — 이번 15개 / 누적 302개
[밑반찬] page 20 완료 — 이번 12개 / 누적 314개
[밑반찬] page 21 완료 — 이번 11개 / 누적 325개
[밑반찬] page 22 완료 — 이번 12개 / 누적 337개
[밑반찬] page 23 완료 — 이번 12개 / 누적 349개
[밑반찬] page 24 완료 — 이번 14개 / 누적 363개
[밑반찬] page 25 완료 — 이번 14개 / 누적 377개
[밑반찬] page 26 완료 — 이번 6개 / 누적 383개
[밑반찬] page 27 완료 — 이번 7개 / 누적 390개
[밑반찬] page 28 완료

KeyboardInterrupt: 

In [2]:
!pip install selenium
!pip install beautifulsoup4
!pip install pandas
!pip install lxml

In [3]:
import re, json, time, random, os
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# ✅ 기본 설정
BASE = "https://www.10000recipe.com"

CAT4_MAP = {
    63: "밑반찬", 56: "메인반찬", 54: "국/탕", 55: "찌개",
    53: "면/만두", 52: "밥/죽/떡", 61: "퓨전", 65: "양식",
    64: "샐러드", 68: "스프", 62: "기타"
}

# ✅ Chrome 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(options=chrome_options)

def get_soup(url):
    """Selenium 기반 BeautifulSoup 파서"""
    driver.get(url)
    time.sleep(random.uniform(1.8, 2.8))  # JS 로딩 대기
    html = driver.page_source
    return BeautifulSoup(html, "html.parser")

# ✅ 조회수 5만 이상 레시피 ID 추출
def get_recipe_ids_over100k(cat4, cat_name):
    ids = []
    page = 1
    prev_ids = set()

    while True:
        url = f"{BASE}/recipe/list.html?cat4={cat4}&order=reco&page={page}"
        soup = get_soup(url)
        cards = soup.select("ul.common_sp_list_ul li")

        if not cards:
            print(f"[{cat_name}] 페이지 없음 — 누적 {len(ids)}개")
            break

        new_ids = []
        for li in cards:
            link = li.select_one("a.common_sp_link")
            if not link or "href" not in link.attrs:
                continue

            m = re.search(r"/recipe/(\d+)", link["href"])
            if not m:
                continue
            recipe_id = int(m.group(1))

            # ✅ 조회수 추출
            views_el = li.select_one("span.common_sp_caption_buyer")
            if not views_el:
                continue
            text = views_el.get_text(strip=True)
            text = text.replace("조회수", "").replace(",", "").replace(" ", "")
            if "만" in text:
                try:
                    views = float(text.replace("만", "")) * 10000
                except:
                    continue
            else:
                try:
                    views = int(re.sub(r"\D", "", text))
                except:
                    continue

            if views >= 50000:
                new_ids.append(recipe_id)

        ids_set = set(new_ids)
        if not new_ids or ids_set == prev_ids:
            print(f"[{cat_name}] 중단 (page {page}) — 새로운 레시피 없음")
            break

        prev_ids = ids_set
        ids.extend(new_ids)
        print(f"[{cat_name}] page {page} 완료 — 이번 {len(new_ids)}개 / 누적 {len(ids)}개")
        page += 1
        time.sleep(random.uniform(1.2, 2.2))

    return list(dict.fromkeys(ids))

# ✅ 레시피 상세 파서
def parse_recipe(recipe_id, cat4=None):
    url = f"{BASE}/recipe/{recipe_id}"
    try:
        soup = get_soup(url)
    except Exception as e:
        print(f"❌ {recipe_id} 불러오기 실패: {e}")
        return None

    title = soup.select_one("div.view2_summary h3, h3.view2_title")
    title = title.get_text(strip=True) if title else None

    summary_el = soup.select_one("#relationGoods > div.best_tit > b:nth-child(1)")
    summary = summary_el.get_text(strip=True) if summary_el else None

    # 인분
    serving_el = soup.select_one(
        "#contents_area_full div.view2_summary_info span.view2_summary_info1"
    )
    servings = None
    if serving_el:
        nums = re.findall(r"\d+", serving_el.get_text(strip=True))
        if nums:
            servings = int(nums[0])

    # 조리시간
    time_el = soup.select_one(
        "#contents_area_full div.view2_summary_info span.view2_summary_info2"
    )
    cook_time = None
    if time_el:
        nums = re.findall(r"\d+", time_el.get_text(strip=True))
        if nums:
            cook_time = int(nums[0])

    # 난이도
    level_el = soup.select_one(
        "#contents_area_full div.view2_summary_info span.view2_summary_info3"
    )
    level = level_el.get_text(strip=True) if level_el else None

    type_nm = CAT4_MAP.get(cat4, None)

    # 재료
    ingredients = {}
    for li in soup.select(".ready_ingre3 li, .ingre_list li"):
        text = li.get_text(" ", strip=True)
        if not text:
            continue
        clean_text = text.replace("구매", "").strip()
        parts = clean_text.split()
        if not parts:
            continue
        name = parts[0]
        amount = " ".join(parts[1:]) if len(parts) > 1 else ""
        ingredients[name] = amount

    # 조리 단계
    steps = []
    for i, el in enumerate(soup.select("#stepDiv .media-body, .view_step_cont"), 1):
        txt = el.get_text(" ", strip=True)
        if txt:
            steps.append(f"{i}. {txt}")

    # 팁
    tips = []
    for i in range(1, len(steps) + 1):
        tip_el = soup.select_one(f"#stepdescr{i} > p")
        tip = tip_el.get_text(strip=True) if tip_el else ""
        tips.append(tip)

    return {
        "RECIPE_ID": recipe_id,
        "RECIPE_NM_KO": title,
        "SUMRY": summary,
        "SERVINGS": servings,
        "TY_NM": type_nm,
        "COOKING_TIME": cook_time,
        "LEVEL_NM": level,
        "INGREDIENT_FULL": json.dumps(ingredients, ensure_ascii=False).replace('"', "'"),
        "STEP_TEXT": json.dumps(steps, ensure_ascii=False),
        "STEP_TIP": json.dumps(tips, ensure_ascii=False),
    }

# ✅ 실행부
if __name__ == "__main__":
    for cat4, cat_name in CAT4_MAP.items():
        print(f"\n🌿 [START] {cat_name} (cat4={cat4}) 수집 시작")
        ids = get_recipe_ids_over100k(cat4, cat_name)
        print(f"[{cat_name}] 조회수 10만 이상 레시피 {len(ids)}개 발견")

        data_list = []
        for rid in ids:
            data = parse_recipe(rid, cat4=cat4)
            if data:
                data_list.append(data)
            time.sleep(random.uniform(1.2, 2.0))

        if data_list:
            df = pd.DataFrame(data_list)
            safe_cat = re.sub(r'[<>:"/\\|?*]', '_', cat_name)
            filename = f"recipes_over100k2_{safe_cat}.csv"
            out_path = os.path.join(os.getcwd(), filename)
            df.to_csv(out_path, index=False, encoding="utf-8-sig")
            print(f"💾 {cat_name} 완료 — {len(df)}개 저장 → {out_path}")
        else:
            print(f"⚠️ {cat_name}: 수집된 데이터 없음")

        time.sleep(2.5)

    driver.quit()
    print("\n🎯 전체 cat4 수집 완료 — 모든 조회수 10만 이상 레시피 저장 완료!")



🌿 [START] 밑반찬 (cat4=63) 수집 시작
[밑반찬] page 1 완료 — 이번 8개 / 누적 8개
[밑반찬] page 2 완료 — 이번 10개 / 누적 18개
[밑반찬] page 3 완료 — 이번 19개 / 누적 37개
[밑반찬] page 4 완료 — 이번 17개 / 누적 54개
[밑반찬] page 5 완료 — 이번 21개 / 누적 75개
[밑반찬] page 6 완료 — 이번 22개 / 누적 97개
[밑반찬] page 7 완료 — 이번 20개 / 누적 117개
[밑반찬] page 8 완료 — 이번 12개 / 누적 129개
[밑반찬] page 9 완료 — 이번 20개 / 누적 149개
[밑반찬] page 10 완료 — 이번 15개 / 누적 164개
[밑반찬] page 11 완료 — 이번 17개 / 누적 181개
[밑반찬] page 12 완료 — 이번 16개 / 누적 197개
[밑반찬] page 13 완료 — 이번 11개 / 누적 208개
[밑반찬] page 14 완료 — 이번 22개 / 누적 230개
[밑반찬] page 15 완료 — 이번 15개 / 누적 245개
[밑반찬] page 16 완료 — 이번 18개 / 누적 263개
[밑반찬] page 17 완료 — 이번 13개 / 누적 276개
[밑반찬] page 18 완료 — 이번 11개 / 누적 287개
[밑반찬] page 19 완료 — 이번 15개 / 누적 302개
[밑반찬] page 20 완료 — 이번 12개 / 누적 314개
[밑반찬] page 21 완료 — 이번 11개 / 누적 325개
[밑반찬] page 22 완료 — 이번 12개 / 누적 337개
[밑반찬] page 23 완료 — 이번 12개 / 누적 349개
[밑반찬] page 24 완료 — 이번 14개 / 누적 363개
[밑반찬] page 25 완료 — 이번 14개 / 누적 377개
[밑반찬] page 26 완료 — 이번 6개 / 누적 383개
[밑반찬] page 27 완료 — 이번 7개 / 누적 390개
[밑반찬] page 28 완료

KeyboardInterrupt: 

In [1]:
from bs4 import BeautifulSoup
print("BeautifulSoup import 성공!")


BeautifulSoup import 성공!
